In [2]:
import pandas as pd
import yfinance as yf
import numpy as np
import ta
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, kpss
import datetime as dt
import sys

In [7]:
class backtest:
    
    def __init__(self, symbol, start):
        self.symbol = symbol
        self.start = start
        self.df = yf.download(self.symbol, start=self.start)
        if self.df.empty:
            print('No pulled data')
        else:
            self.keltner_band()
            self.signals()
            self.loop_bt()
            print(self.calc_profit())
            self.calc_buyhold
            self.buyHold = self.calc_buyhold()
            #self.profit = self.calc_profit()
            #self.cum_profit = (self.profit + 1).prod() - 1 
            #self.plot_bands()
            #self.plot_signals()
    
    def keltner_band(self):
        self.df['HighBand'] = ta.volatility.keltner_channel_hband(self.df.High, self.df.Low, self.df.Close, original_version=False)
        self.df['MiddleBand'] = ta.volatility.keltner_channel_mband(self.df.High, self.df.Low, self.df.Close, original_version=False)
        self.df['LowBand'] = ta.volatility.keltner_channel_lband(self.df.High, self.df.Low, self.df.Close, original_version=False)
        self.df['rsi'] = ta.momentum.rsi(self.df.Close, window=6)
        self.df['shifted_close'] = self.df.Close.shift()
        
    def plot_bands(self):
        plt.figure(figsize=(15,5))
        plt.plot(self.df['2022':][['Close', 'HighBand', 'MiddleBand', 'LowBand']], label=['Close', 'Upper', 'Middle', 'Lower'])
        #plt.fill_between(df.index, df.HighBand, df.LowBand, color='grey', alpha=0.3)
        plt.legend(loc='lower left') 
        
    def signals(self):
        conditions = [(self.df.rsi < 30) & (self.df.Close < self.df.LowBand),
                      (self.df.rsi > 70) & (self.df.Close > self.df.HighBand)]
        choices = ['Buy', 'Sell']
        self.df['signal'] = np.select(conditions, choices)
        self.df.signal = self.df.signal.shift()
        self.df.dropna(inplace=True)
        
    def plot_signals(self):
        plt.figure(figsize=(12, 5))
        plt.plot(self.df.Open, label = 'Open Price')
        plt.scatter(self.buy_arr.index, self.buy_arr.values, marker='^', color='g', label = 'Buy')
        plt.scatter(self.sell_arr.index, self.sell_arr.values, marker='v', color='r', label = 'Sell')
        plt.legend()
        
    def current_signal(self):
        from termcolor import colored
        signal = self.df.signal[len(self.df) - 1]
        print(f'Current signal as of {self.df.index[len(self.df) - 1]}:\n')
        print(colored(signal, 'yellow', attrs=['bold']))
        
    def loop_bt(self):
        position = False
        buydate, selldate = [], []

        for index, row in self.df.iterrows():
            if not position:
                if row['signal'] == 'Buy':
                    buydate.append(index)
                    position = True
            if position:
                if row['signal'] == 'Sell' or row['shifted_close'] < 0.95 * self.df.loc[buydate[-1]].Open:
                    selldate.append(index)
                    position = False
            
        self.buy_arr = self.df.loc[buydate].Open
        self.sell_arr = self.df.loc[selldate].Open
        
    def calc_profit(self):
        if self.buy_arr.index[-1] > self.sell_arr.index[-1]:
            self.buy_arr = self.buy_arr[:-1]
            return (self.sell_arr.values - self.buy_arr.values) / self.buy_arr.values
        
        #try:
        #    if self.buy_arr.index[-1] > self.sell_arr.index[-1]:
        #        self.buy_arr = self.buy_arr[:-1]
        #        return (self.sell_arr.values - self.buy_arr.values) / self.buy_arr.values
        #except:
        #    print('No buy/sell signals')
        #    print(f'Buy: {self.buy_arr}\nSell: {self.sell_arr}')
    
    def calc_buyhold(self):
        buyHold = (self.df['Adj Close'].pct_change() + 1).prod() - 1
        return buyHold   

In [8]:
bt = backtest('MSFT', '2020-12-31')

print('------   RESULTS   -----')
print(f'Strategy: {round(bt.cum_profit,4)*100}%\nBuy/hold: {round(bt.buyHold,4)*100}%\nDifference: {round(bt.cum_profit-bt.buyHold,4)*100}%')

[*********************100%***********************]  1 of 1 completed
None
------   RESULTS   -----


AttributeError: 'backtest' object has no attribute 'cum_profit'

In [10]:
print((bt.sell_arr.values - bt.buy_arr.values) / bt.buy_arr.values)

[-0.07179814 -0.04821172 -0.08078596  0.27413986 -0.08406665 -0.05699096
  0.21652512  0.25698859]


---

In [ ]:
class momentum:
    
    def __init__(self):
        self.get_symbols()
        self.get_rolling_ret()
        
    def get_symbols(self): # NSB model
        self.component = pd.read_html('https://siblisresearch.com/data/historical-components-nasdaq/')[0]
        self.component.set_index('Ticker', inplace=True)
        self.component = self.component[self.component.columns[2:]]
        self.component.columns = pd.to_datetime(self.component.columns)
        self.component = self.component == 'X'
        tickers = self.component.index.to_list()
        self.df = yf.download(tickers, start='2016-01-01')['Adj Close']
        self.df = self.df.dropna(axis=1, how='all')
        self.component = self.component.loc[self.component.index.isin(self.df.columns)]
        self.df.index = pd.to_datetime(self.df.index)
        self.mtl = (self.df.pct_change() + 1)[1:].resample('M').prod()
        #self.mtl.head()
        return self.component
        
    def get_rolling_ret(self):
        self.ret_12 = self.mtl.rolling(12).apply(np.prod)
        self.ret_6 = self.mtl.rolling(6).apply(np.prod)
        self.ret_3 = self.mtl.rolling(3).apply(np.prod)
        return self.ret_12, self.ret_6, self.ret_3
    
    def get_relevant(self, date):
        self.date = date
        ix = np.argmax(pd.to_datetime(date) >= self.component.columns)
        return self.component[self.component.iloc[:,ix]].index
    
    def filter_ret(self, date, df):
        self.date = date
        self.df = df
        return self.df[self.get_relevant(date)]
    
    def get_top(self, date):
        self.date = date
        ret_12m, ret_6m, ret_3m = self.filter_ret(date,self.ret_12), self.filter_ret(date,self.ret_6), self.filter_ret(date,self.ret_3)
        
        top_50 = ret_12m.loc[date].nlargest(50).index
        top_30 = ret_6m.loc[date].nlargest(30).index
        top_10 = ret_3m.loc[date].nlargest(10).index
        return list(top_10)

In [ ]:
mom = momentum()

In [ ]:
mom.get_top('2022-12-31')

---

In [ ]:
bt.plot_bands()

---

In [ ]:
class metrics:
    
    def __init__(self, symbol, start):
        self.symbol = symbol
        self.start = start
        self.instance = backtest(symbol, start)
        
    def historical_metrics(self):
        returns = (self.df['Adj Close'].pct_change())
        #volatility
        ann_volatility = returns.std()*(252**0.5)
        mon_volatility = returns.std()*(21**0.5)  
        #sharpe
        sharpe = (((returns + 1).prod()**(252/self.df.shape[0])-1) - 0.03) / ann_volatility
        #drawdown

        print(f'Annaual Vol: {round(ann_volatility,2)}\nMonthly Vol: {round(mon_volatility,2)}\nAnnual Sharpe Ratio: {sharpe}')

    def stationarity(self):
        import warnings
        from statsmodels.tools.sm_exceptions import InterpolationWarning
        warnings.simplefilter('ignore', InterpolationWarning)

        adf = adfuller(self.instance.df['Adj Close'])
        kpss_test = kpss(self.instance.df['Adj Close'], regression="c", nlags="auto")

        print(f'ADF:\nTest-value: {adf[0]}\np-value: {adf[1]}\n')
        print(f'KPSS:\nTest-value {kpss_test[0]}\np-value: {kpss_test[1]}')

In [ ]:
ins = metrics('^GSPC', '2017-01-01')

In [ ]:
ins.stationarity()